In [1]:
import numpy as np

# Constants
hbar = 1.0545718e-34  # Planck's constant (J*s)
pi = np.pi

# Input Parameters
num_states = 10  # Number of electronic states
num_modes = 5    # Number of phonon modes

# Example data (replace with actual values)
E = np.random.rand(num_states)  # Energy levels (eV)
eigenvectors = np.random.rand(num_states, num_states)  # Eigenvector matrix
V_alpha_matrix = np.random.rand(num_states, num_states, num_modes)  # Operator in eigenbasis
n_alpha = np.random.rand(num_modes)  # Phonon occupation numbers
omega_alpha = np.random.rand(num_modes)  # Phonon frequencies (eV)

# Define delta function as a small Gaussian
def delta(x, epsilon=1e-5):
    return np.exp(-(x**2) / (2 * epsilon**2)) / (np.sqrt(2 * pi) * epsilon)

# Compute matrix element <a|V_alpha|b>
def V_alpha_element(a, b, alpha):
    return np.dot(eigenvectors[a], V_alpha_matrix[:, :, alpha].dot(eigenvectors[b]))

# Compute W_ab terms
def compute_W_terms(a, b, alpha, beta):
    W_pm = 0
    W_mp = 0
    W_pp = 0
    W_mm = 0

    for c in range(num_states):
        # Energy differences
        E_ac = E[a] - E[c]
        E_bc = E[b] - E[c]

        # Matrix elements
        V_alpha_ac = V_alpha_element(a, c, alpha)
        V_alpha_cb = V_alpha_element(c, b, beta)

        # Terms for W_ab^{+-}
        term1 = np.abs(V_alpha_ac * V_alpha_cb)**2
        term2 = n_alpha[alpha] * (n_alpha[beta] + 1)
        term3 = delta(E_bc - hbar * omega_alpha[alpha] + hbar * omega_alpha[beta])
        W_pm += term1 * term2 * term3 / E_ac

        # Terms for W_ab^{-+}
        term4 = np.abs(V_alpha_ac * V_alpha_cb)**2
        term5 = (n_alpha[alpha] + 1) * n_alpha[beta]
        term6 = delta(E_bc - hbar * omega_alpha[alpha] - hbar * omega_alpha[beta])
        W_mp += term4 * term5 * term6 / E_ac

        # Terms for W_ab^{++}
        term7 = np.abs(V_alpha_ac * V_alpha_cb)**2
        term8 = (n_alpha[alpha] + 1) * (n_alpha[beta] + 1)
        term9 = delta(E_bc + hbar * omega_alpha[alpha] + hbar * omega_alpha[beta])
        W_pp += term7 * term8 * term9 / E_ac

        # Terms for W_ab^{--}
        term10 = np.abs(V_alpha_ac * V_alpha_cb)**2
        term11 = n_alpha[alpha] * n_alpha[beta]
        term12 = delta(E_bc - hbar * omega_alpha[alpha] - hbar * omega_alpha[beta])
        W_mm += term10 * term11 * term12 / E_ac

    return W_pm, W_mp, W_pp, W_mm

# Compute W_ab terms for all alpha, beta pairs
W_pm_total = np.zeros((num_states, num_states))
W_mp_total = np.zeros((num_states, num_states))
W_pp_total = np.zeros((num_states, num_states))
W_mm_total = np.zeros((num_states, num_states))

for alpha in range(num_modes):
    for beta in range(num_modes):
        for a in range(num_states):
            for b in range(num_states):
                W_pm, W_mp, W_pp, W_mm = compute_W_terms(a, b, alpha, beta)
                W_pm_total[a, b] += W_pm
                W_mp_total[a, b] += W_mp
                W_pp_total[a, b] += W_pp
                W_mm_total[a, b] += W_mm

# Print example results
print("W_ab^{+-}:", W_pm_total)
print("W_ab^{-+}:", W_mp_total)
print("W_ab^{++}:", W_pp_total)
print("W_ab^{--}:", W_mm_total)

W_minus_minus:
[[161031.08454557  32569.17971386 102733.36040916]
 [ 65134.33238512 256976.03262485    600.68348989]
 [   975.54145615 154338.31560783 245624.15162705]]

W_plus_plus:
[[1354692.43181837  258614.91828994  871306.12530797]
 [ 555060.05123232 2105258.35405674    4430.80413109]
 [   7891.73277266 1301903.14554166 1894028.69670618]]

W_plus_minus:
[[467062.72760882  96380.13537723 290455.70736659]
 [192292.22908939 735527.66057833   1646.45808719]
 [  2801.86661564 457383.24063534 682069.78512887]]

W_minus_plus:
[[467062.72760882  87392.23821905 308178.50683511]
 [188013.14042628 735527.66057833   1616.50691822]
 [  2747.7084161  439311.10875056 682069.78512887]]

